#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [3]:
#import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import plotly.express as px
from scipy import stats
from scipy.stats import norm, skew # para estadisticas


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # limitar flotantes a 3 decimales


from subprocess import check_output


import urllib.request
from PIL import Image

## Data

In [54]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv(r'data\train.csv',encoding='latin-1')


## Data exploration

In [55]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73363 entries, 0 to 73362
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id                                    73363 non-null  int64  
 1   target                                73363 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  73363 non-null  float64
 3   age                                   73363 non-null  object 
 4   NumberOfTime30-59DaysPastDueNotWorse  73363 non-null  int64  
 5   DebtRatio                             73363 non-null  float64
 6   MonthlyIncome                         58796 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       73363 non-null  int64  
 8   NumberOfTimes90DaysLate               73363 non-null  int64  
 9   NumberRealEstateLoansOrLines          73363 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  73363 non-null  int64  
 11  NumberOfDepende

In [56]:
train.head()

,Id,target,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,44164,0,0.768,68y,1,0.688,2178.000,12,0,1,0,0.000
1,95356,0,0.290,52y,0,0.385,3750.000,16,0,1,0,1.000
2,108917,0,0.000,54y,0,0.347,6833.000,8,0,1,0,0.000
3,99039,0,0.046,66y,1,0.408,5200.000,6,0,1,0,1.000
4,116168,0,0.169,45y,0,4702.000,NaN,10,0,3,0,1.000


## Data processing

In [57]:
train['age'] = train['age'].apply(lambda x: str(x).split('y')[0])
train['age'] = train['age'].apply(int)

In [58]:
train['MonthlyIncome'].fillna(train['MonthlyIncome'].mean(), inplace=True)
train.dropna(inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71421 entries, 0 to 73362
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id                                    71421 non-null  int64  
 1   target                                71421 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  71421 non-null  float64
 3   age                                   71421 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  71421 non-null  int64  
 5   DebtRatio                             71421 non-null  float64
 6   MonthlyIncome                         71421 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       71421 non-null  int64  
 8   NumberOfTimes90DaysLate               71421 non-null  int64  
 9   NumberRealEstateLoansOrLines          71421 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  71421 non-null  int64  
 11  NumberOfDependents  

In [59]:
train1 = train[~((train['age'] > 98) & (train['age'] < 20))].copy()
train1 = train1[~(train1['NumberOfTime30-59DaysPastDueNotWorse'] > 80)]

In [62]:
train2 = train1.drop(columns = ['Id','NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents'])

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [63]:
X = train2.iloc[:,1:]
y = train2['target']

In [64]:
X.head()

,RevolvingUtilizationOfUnsecuredLines,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines
0,0.768,68,0.688,2178.000,12,0,1
1,0.290,52,0.385,3750.000,16,0,1
2,0.000,54,0.347,6833.000,8,0,1
3,0.046,66,0.408,5200.000,6,0,1
4,0.169,45,4702.000,6649.145,10,0,3


In [65]:
X.shape

(71308, 7)

In [66]:
y.shape

(71308,)

### 2. Dividir X_train, X_test, y_train, y_test

In [14]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import lightgbm as lgb


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [68]:
X_train

,RevolvingUtilizationOfUnsecuredLines,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines
67554,0.000,65,0.210,5946.000,8,0,1
49620,0.000,46,6220.000,6649.145,7,0,2
24922,0.066,53,0.653,4416.000,10,0,1
31065,0.170,74,0.203,9050.000,12,0,1
39710,0.428,39,0.320,5100.000,7,0,1
...,...,...,...,...,...,...,...
38248,0.113,38,0.412,11230.000,7,0,3
6451,1.000,56,0.275,5700.000,7,0,1
56460,1.000,42,0.019,22710.000,2,1,0
890,0.000,59,1475.000,6649.145,9,0,1


### 3. Asignar el modelo (vacío) a una variable


In [17]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(model.mean(), model.std()))

Gradient Boosting score: 0.2568 (0.0058)



In [69]:
model = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [70]:
model.fit(X,y)

GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=4,
                          max_features='sqrt', min_samples_leaf=15,
                          min_samples_split=10, n_estimators=3000,
                          random_state=5)

In [71]:
pred = model.predict(X_test)
pred

array([0., 0., 0., ..., 0., 0., 0.])

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [72]:
from sklearn.metrics import mean_squared_error
import numpy as np

print('RMSE :',np.sqrt(mean_squared_error(y_test,pred)))

RMSE : 0.25618146702507233


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [122]:
X_pred = pd.read_csv("data/test.csv")
X_pred.head()

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,15576,0.052,53y,0,0.009,2245.000,3,0,0,0,0.000
1,94308,0.022,57y,0,0.320,7672.000,26,0,1,0,2.000
2,53983,0.014,68y,0,0.139,4064.000,4,0,0,0,1.000
3,61359,0.019,74y,0,0.019,10500.000,27,0,0,0,0.000
4,71383,0.009,66y,0,9.000,NaN,5,0,0,0,0.000


In [131]:
X_pred['age'] = X_pred['age'].apply(lambda x: str(x).split('y')[0])
X_pred['age'] = X_pred['age'].apply(int)
X_pred['MonthlyIncome'].fillna(X_pred['MonthlyIncome'].mean(), inplace=True)
X_pred1 = X_pred[~((X_pred['age'] > 98) & (X_pred['age'] < 20))].copy()
X_pred2 = X_pred1.drop(columns = ['Id','NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents'])


In [148]:
X_pred2.shape

(31442, 7)

In [133]:
X_pred2

,RevolvingUtilizationOfUnsecuredLines,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines
0,0.052,53,0.009,2245.000,3,0,0
1,0.022,57,0.320,7672.000,26,0,1
2,0.014,68,0.139,4064.000,4,0,0
3,0.019,74,0.019,10500.000,27,0,0
4,0.009,66,9.000,6766.742,5,0,0
...,...,...,...,...,...,...,...
31437,0.469,28,0.029,2166.000,1,0,0
31438,0.172,44,0.376,5833.000,8,0,2
31439,0.005,51,30.000,0.000,3,0,0
31440,0.637,42,0.484,7000.000,10,2,1


**¿Por qué da error?** 

In [147]:
predictions_submit = model.predict(X_pred2)
predictions_submit

array([0., 0., 0., ..., 0., 0., 0.])

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [144]:
predictions_submit = model.predict(X_pred2)
predictions_submit

array([0., 0., 0., ..., 0., 0., 0.])

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [149]:
sample = pd.read_csv("data/sample_submission.csv")

In [150]:
sample.head()

,Id,target
0,15576,0
1,94308,1
2,53983,1
3,61359,1
4,71383,0


In [151]:
sample.shape

(31442, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [153]:
submission = pd.DataFrame({"Id": sample['Id'], "target": predictions_submit})

In [154]:
submission.head()

,Id,target
0,15576,0.000
1,94308,0.000
2,53983,0.000
3,61359,0.000
4,71383,0.000


In [155]:
submission.shape

(31442, 2)

In [114]:
submission.to_csv('submission.csv', index=False)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [112]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [113]:
chequeator(submission)

Check the number of rows and/or columns and try again

Mensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.
